In [ ]:
import os
import wget

In [ ]:
links = {
    'protobuff': "https://github.com/protocolbuffers/protobuf/releases/download/v25.0-rc1/protoc-25.0-rc-1-linux-x86_64.zip",
    'tensorflow_models': "https://github.com/tensorflow/models/archive/refs/heads/master.zip",
    'pretrained_model': "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz",
    'tfrecord_script': "https://github.com/afaqueumer/TensorFlow-Object-Detection/raw/main/generate_tfrecord.zip"
}

In [ ]:
root_path = "/home/bzy/TensorFlow"
new_model_name = 'custom_cube_detect' 
pretrained_model = 'ssd_mobilenet_v2_320x320_coco17_tpu-8'
tf_record_script = 'generate_tfrecord.py'
label_map = 'label_map.pbtxt'

In [ ]:
paths = {
    'tensorflow': root_path,    
    'workspace': os.path.join(root_path, 'workspace'),
    'scripts': os.path.join(root_path,'scripts'),
    'api': os.path.join(root_path,'models'),
    'annotation': os.path.join(root_path, 'workspace','annotations'),
    'image': os.path.join(root_path, 'workspace','images'),
    'train_images': os.path.join(root_path, 'workspace','images','train'),
    'test_images': os.path.join(root_path, 'workspace','images','test'),
    'model': os.path.join(root_path, 'workspace','models'),
    'pretrained_model': os.path.join(root_path, 'workspace','pre-trained-models'),
    'checkpoint': os.path.join(root_path, 'workspace','models', new_model_name), 
    'output': os.path.join(root_path, 'workspace','models', new_model_name, 'export'), 
    'protoc':os.path.join(root_path,'protoc')
 }

for path in paths.values():
    if not os.path.exists(path):
        os.mkdir(path)

In [ ]:
files = {
    'train_pipeline':os.path.join('/home/bzy/tf', 'workspace','models', new_model_name, 'pipeline.config'),
    'tf_record_script': os.path.join(paths['scripts'], tf_record_script), 
    'labelsmap': os.path.join(paths['annotation'], label_map)
}

In [ ]:
wget.download(links["protobuff"], paths['protoc'])
!cd {paths['protoc']} && unzip "protoc*.zip"

os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['protoc'], 'bin')) 

In [ ]:
wget.download(links["tensorflow_models"], paths['api'])
!cd {paths['api']} && unzip models-master.zip
!cd {paths['api']} && cp -r ./models-master/. ./.
!cd {paths['api']} && rm -rf models-master

In [ ]:
wget.download(links["pretrained_model"], paths['pretrained_model'])
!cd {paths['pretrained_model']} && tar -zxvf {pretrained_model + '.tar.gz'}


In [ ]:

wget.download(links["tfrecord_script"], paths['scripts'])
!cd {paths['scripts']} && unzip generate_tfrecord.zip

In [ ]:
object_detection = os.path.join(paths['api'], 'research', 'object_detection')

In [ ]:
!cd {os.path.join(paths['api'], 'research')} && protoc object_detection/protos/*.proto --python_out=.

In [ ]:
!cd {object_detection} && rm -rf cocoapi && git clone https://github.com/cocodataset/cocoapi.git
!cd {object_detection}/cocoapi/PythonAPI && make
!cd {object_detection}/cocoapi/PythonAPI && cp -r pycocotools {paths['api']}/research/

In [ ]:
!cd {os.path.join(paths['api'], 'research')} && cp object_detection/packages/tf2/setup.py .
!cd {os.path.join(paths['api'], 'research')} && python -m pip install .


# !cd {os.path.join(paths['api'], 'research')} && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py setup.py && python setup.py build && python setup.py install
# !cd {os.path.join(paths['api'], 'research')}/slim && pip install -e 
verification_script = os.path.join(paths['api'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
!python {verification_script}

In [ ]:
import object_detection

In [ ]:
labels = [{'id':1, 'name':'cube'}, {'id':2, 'name':'cube'}]
with open(files['labelsmap'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write(f'\tid:{label["id"]}\n')
        f.write(f'\tname:\'{label["name"]}\'\n')
        f.write('}\n')

In [ ]:
!python {files['tf_record_script']} -x {paths['train_images']} -l {files['labelsmap']} -o {os.path.join(paths['annotation'], 'train.record')} 
!python {files['tf_record_script']} -x {paths['test_images']} -l {files['labelsmap']} -o {os.path.join(paths['annotation'], 'test.record')} 

In [ ]:
!cp -i {os.path.join(paths['pretrained_model'], pretrained_model, 'pipeline.config')} {os.path.join(paths['checkpoint'])}

In [ ]:
!cp -i {os.path.join(paths['api'], 'research', 'object_detection', 'model_main_tf2.py')} {paths['scripts']}

In [ ]:
# training_script = 'model_main_tf2.py'
# training_command = "python {} --model_dir=/models/{} --pipeline_config=models/{}/pipeline.config".format(training_script, new_model_name, new_model_name)
training_command = f"cd {paths['workspace']} && python model_main_tf2.py --model_dir=models/{new_model_name} --pipeline_config_path=models/{new_model_name}/pipeline.config"
! pip install tensorflow==2.13.0

## Train

In [ ]:

!{training_command}

## Evaluating the Model

In [ ]:
evaluating_command = f"cd {paths['workspace']} && python model_main_tf2.py --model_dir=models/{new_model_name} --pipeline_config_path=models/{new_model_name}/pipeline.config --checkpoint_dir=models/{new_model_name}"
!{evaluating_command}

## Monitor

In [ ]:
monitoring_command = f"cd {paths['workspace']} && tensorboard --logdir=models/{new_model_name}"
!{monitoring_command}

## Export

In [ ]:

!cp -i {os.path.join(paths['api'], 'research', 'object_detection', 'exporter_main_v2.py')} {paths['scripts']}

In [ ]:
exporting_script = os.path.join(paths['scripts'], 'exporter_main_v2.py')
exporting_command = "python {} --input_type image_tensor --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(exporting_script, files['train_pipeline'], paths['checkpoint'], paths['output'])

In [ ]:
!{exporting_command}